In [ ]:
%%html
<style>
    #notebook-container{
    width: 100%
    }
</style>

In [ ]:
import altair as alt
import pandas as pd

from altair import datum
from vega_datasets import data
from geopy.geocoders import Nominatim

# alt.renderers.enable('notebook')

In [ ]:
alt.data_transformers.enable('default', max_rows=None) 

In [ ]:
# df = pd.read_csv('https://raw.githubusercontent.com/kai-wren/UCU_DataViz/master/gapminder.csv')
df = pd.read_csv('scrubbed_UFO.csv')
df = df.rename(columns={'longitude ': 'longitude'})
df.replace(to_replace='&#39', value='\'', inplace=True, regex=True)
df.replace(to_replace='&#44', value=',', inplace=True, regex=True)
df['type'] = "UFO"

df.head()

In [ ]:
df_meteorite = pd.read_csv('meteorite-landings.csv')
df_meteorite['type'] = "Meteorite"

df_meteorite.head()

Question to be answered: <br>
Are there places on Earth which have more UFOs spotted than in other places. <br>

Here I am plotting a point indicating one record each. This allow to see how encounters are spread on world map. Points are located based on latitude and longitude values. Hence if those values are empty - points located wrongly in the top left corner. In the end density of points indicates places with bigger amounts of contacts. <br>

For interactivity I've added barchart with interval selection to limited amount of dots to be displayed.

In [ ]:
base_ufo = alt.Chart(df)

source = alt.topo_feature(data.world_110m.url, 'countries')

# slider = alt.binding_range(min=1900, max=2015, step=1, name='Year')
# selector = alt.selection_single(name="YearSelector", fields=['cutoff'],
#                                 bind=slider, init={'cutoff': 2015})

brush = alt.selection_interval(encodings=['x'],empty='all')

base_world = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
)

earth = base_world.project('naturalEarth1')

points = base_ufo.mark_circle(
    size=10,
    color='red'
).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    tooltip=['shape:N', 'city:N', 'duration (hours/min):N', 'date posted:N']
).transform_filter(brush)
# .add_selection(selector).transform_calculate(year = "year(datum.datetime)").transform_filter(datum.year <= selector.cutoff)

area_ufo = base_ufo.mark_bar().encode(
x=alt.X('year(datetime):N'),
y=alt.Y('count()'),
opacity=alt.value(0.3)
)

area_ufo_notselected = area_ufo.add_selection(brush)
area_ufo_selected = area_ufo.mark_bar().encode(opacity=alt.value(1)).transform_filter(brush)

(earth+points).properties(width=1200, height=600) & (area_ufo_notselected + area_ufo_selected).properties(width=1200, height=100)

Question to be answered: <br>
Does the amount of spotted UFO increase with the course of time? This is especially interesting with regards to last 20-30 years with widespread communication technologies. <br>

Here I plot simple barchart to highlight an answer to the question. In addition I've added another chart which allows to see in which month and day how many observation happens. <br>

For interactivity, you could select one particular year or years and right hand side chart will be updated for selected years.

In [ ]:
base_ufo = alt.Chart(df)

selector = alt.selection_multi(encodings=['x'],empty='all')

bars = base_ufo.mark_bar().encode(
y=alt.Y('count()'),
x=alt.X('year(datetime):N'),
color = alt.condition(selector, alt.value('blue'), alt.value('gray'))
).add_selection(selector
).properties(width=1000, height=500)

ticks = base_ufo.mark_point().encode(
y=alt.Y('month(datetime):N'),
x=alt.X('date(datetime):O'),
size=alt.Size('count()')
).transform_filter(selector
).properties(width=600, height=500)

bars | ticks

Question to be answered: <br>
Do types of spotted UFOs equally spread or only appears in certain locations.

I'be made same map, but with intractive legend this time to allow selection of particular shape of UFO spotted. I'm thinking on adding some barcharts, which will indicate some proportions.

In [ ]:
base_ufo = alt.Chart(df)

source = alt.topo_feature(data.world_110m.url, 'countries')

selection = alt.selection_multi(fields=['shape'], bind='legend')

base_world = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
)

earth = base_world.project('naturalEarth1')

points = base_ufo.mark_circle(
    size=10
).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    color=alt.Color('shape:N'),
    tooltip=['shape:N', 'city:N'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0))
).add_selection(
    selection
).properties(width=1200, height=600)

bars = base_ufo.mark_bar().encode(
x=alt.X('shape:N', sort='y'),
y=alt.Y('count()'),
    
).properties(width=600, height=600)

(earth+points)| bars

Question to be answered: <br>
How long UFO were observed per country in the dataset?

I was going to represent each country as bubble on world map size of which indicats how long overall duration per country. Unfortunately, I've discovered that country name is almost missing in the dataset. Hence I will need to enrich it first using some other services. For now Canada and UK indicate example how it should look like.

In [ ]:
base_ufo = alt.Chart(df)

source = alt.topo_feature(data.world_110m.url, 'countries')

base_world = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
)

earth = base_world.project('naturalEarth1')

points = base_ufo.mark_circle(
    size=10
).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    size=alt.Size('duration:Q'),
    tooltip=['country:N']
).transform_aggregate(
latitude='mean(latitude)',
longitude='mean(longitude)',
duration = 'sum(duration (seconds))',
groupby=['country']
)

(earth+points).properties(width=1200, height=600)

# test = base_ufo.mark_bar(
#     size=10
# ).encode(
#     x='country:N',
#     y='sum(duration (seconds)):Q',
# )
# # .transform_aggregate(
# # latitude='mean(latitude)',
# # longitude='mean(longitude)',
# # duration = 'sum(duration (seconds))',
# # groupby=['country']
# # )

# test

Question to be answered: <br>
I would like to match data with meteorite landing dataset to check that maybe some spotted UFOs were just meteorites and whether or not the concentration of spotted UFO coincides with the concentration of observed meteorites on the global map.



In [ ]:
base_meteorite = alt.Chart(df_meteorite)
base_ufo = alt.Chart(df)

source = alt.topo_feature(data.world_110m.url, 'countries')

mouseover = alt.selection_single(on='mouseover', fields=['type'])

base_world = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
)

earth = base_world.project('naturalEarth1')

points_meteorite = base_meteorite.mark_circle(
    size=10,
).encode(
    latitude='reclat:Q',
    longitude='reclong:Q',
    color=alt.condition(mouseover, alt.Color('type:N'), alt.value('gray')),
#     tooltip=['shape:N', 'city:N', 'duration (hours/min):N', 'date posted:N']
    opacity=alt.value(0.5)
).add_selection(mouseover)

points_ufo = base_ufo.mark_circle(
    size=10,
).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    color=alt.condition(mouseover, alt.Color('type:N'), alt.value('gray')),
    tooltip=['shape:N', 'city:N', 'duration (hours/min):N', 'date posted:N'],
    opacity=alt.value(0.5)
)

(earth+points_ufo+points_meteorite).properties(width=1200, height=600)